In [52]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings("ignore")
## https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

In [53]:
df = pd.read_excel(r"Online Retail.xlsx")

In [54]:
df.shape

(541909, 8)

In [55]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
530174,580819,72819,CUPID DESIGN SCENTED CANDLES,12,2011-12-06 11:30:00,1.25,15318.0,United Kingdom
159561,550345,22423,REGENCY CAKESTAND 3 TIER,4,2011-04-18 09:06:00,12.75,15304.0,United Kingdom
432188,573840,21192,WHITE BELL HONEYCOMB PAPER,48,2011-11-01 11:38:00,1.45,14866.0,United Kingdom
100078,544801,22859,EASTER TIN BUNNY BOUQUET,6,2011-02-23 13:48:00,1.65,16145.0,United Kingdom
533399,581023,22891,TEA FOR ONE POLKADOT,2,2011-12-07 10:35:00,8.29,NaN,United Kingdom


In [56]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [57]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [58]:
df.dropna(axis=0, subset=['Description'], inplace=True)

In [59]:
import re
## Removing white space
df['Description'] = df['Description'].astype('str').apply(lambda x: re.sub(r'\s+', ' ', x).strip())

In [60]:
## Sample of cancelled transactions
df[df['InvoiceNo'].astype(str).str.contains('C')].sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
87634,C543671,22602,RETROSPOT WOODEN HEART DECORATION,-36,2011-02-11 11:24:00,0.85,18087.0,United Kingdom
120847,C546680,21391,FRENCH LAVENDER SCENT HEART,-3,2011-03-15 16:51:00,2.10,13121.0,United Kingdom
311271,C564232,85159B,"WHITE TEA,COFFEE,SUGAR JARS",-8,2011-08-24 10:25:00,1.95,13517.0,United Kingdom
465425,C576235,23048,SET OF 10 LANTERNS FAIRY LIGHT STAR,-3,2011-11-14 13:28:00,4.15,17769.0,United Kingdom
323246,C565284,21232,STRAWBERRY CERAMIC TRINKET BOX,-1,2011-09-02 11:31:00,1.25,12471.0,Germany
256308,C559486,22423,REGENCY CAKESTAND 3 TIER,-1,2011-07-08 13:51:00,12.75,13047.0,United Kingdom
233035,C557413,23193,BUFFALO BILL TREASURE BOOK BOX,-1,2011-06-20 11:39:00,2.25,14701.0,United Kingdom
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
88002,C543744,21655,HANGING RIDGE GLASS T-LIGHT HOLDER,-12,2011-02-11 13:43:00,1.69,13672.0,United Kingdom
352782,C567690,20725,LUNCH BAG RED RETROSPOT,-9,2011-09-21 17:01:00,1.65,15810.0,United Kingdom


In [61]:
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
df =df[~df['InvoiceNo'].str.contains('C')]

In [62]:
df.shape

(531167, 8)

In [63]:
print("Total Number of Countries:", df['Country'].nunique(), end="\n\n")
df['Country'].value_counts(normalize=True).head()

Total Number of Countries: 38



Country
United Kingdom    0.915283
Germany           0.017023
France            0.015829
EIRE              0.014862
Spain             0.004678
Name: proportion, dtype: float64

In [64]:
#focussing on one Geography
grouped_df = (df[df['Country']=='Germany']
              .groupby(['InvoiceNo', 'Description'])
               .size()
               .unstack()
               .fillna(0)
               .reset_index()
               .set_index('InvoiceNo'))


print(grouped_df.shape)
grouped_df.head()

(457, 1693)


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC STAR T-LIGHT HOLDER,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
def custom_encoding(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
grouped_df= grouped_df.applymap(custom_encoding)
grouped_df.drop('POSTAGE', inplace=True, axis=1)

In [71]:
frequent_itemsets = apriori(grouped_df, min_support=0.05, use_colnames=True)


In [72]:
rules = association_rules(frequent_itemsets, len(frequent_itemsets), metric="confidence", min_threshold=0.6)

In [73]:
rules.sort_values(by ='lift', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,1.0,0.050194,5.587746,0.913551,0.428571,0.821037,0.654634
1,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,1.0,0.092679,4.529540,0.837922,0.483871,0.779227,0.684524
2,(SPACEBOY LUNCH BOX),(ROUND SNACK BOXES SET OF4 WOODLAND),0.102845,0.245077,0.070022,0.680851,2.778116,1.0,0.044817,2.365427,0.713415,0.251969,0.577243,0.483283
